[Reference](https://towardsdatascience.com/time-series-prediction-beyond-test-data-3f4625019fd9)

In [1]:
pip install chart-studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 1.1 MB/s 


In [2]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")

from chart_studio import plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
pyoff.init_notebook_mode(connected=True)

In [3]:
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

df = pd.read_csv('https://raw.githubusercontent.com/abaranovskis-redsamurai/automation-repo/master/forecast-lstm/shampoo.csv', parse_dates=[0], index_col=0, date_parser=parser)
df.tail()

,Sales
Month,
1903-08-01,407.6
1903-09-01,682.0
1903-10-01,475.3
1903-11-01,581.3
1903-12-01,646.9


In [4]:
train = df

scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)

In [5]:
n_input = 12
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)

In [6]:
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')

In [7]:
history = model.fit_generator(generator,epochs=100,verbose=1)

Epoch 1/100
4/4 [==============================] - 3s 35ms/step - loss: 0.2558
Epoch 2/100
4/4 [==============================] - 0s 40ms/step - loss: 0.1832
Epoch 3/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1214
Epoch 4/100
4/4 [==============================] - 0s 18ms/step - loss: 0.0590
Epoch 5/100
4/4 [==============================] - 0s 18ms/step - loss: 0.0233
Epoch 6/100
4/4 [==============================] - 0s 18ms/step - loss: 0.0416
Epoch 7/100
4/4 [==============================] - 0s 19ms/step - loss: 0.0229
Epoch 8/100
4/4 [==============================] - 0s 18ms/step - loss: 0.0252
Epoch 9/100
4/4 [==============================] - 0s 20ms/step - loss: 0.0262
Epoch 10/100
4/4 [==============================] - 0s 20ms/step - loss: 0.0222
Epoch 11/100
4/4 [==============================] - 0s 21ms/step - loss: 0.0217
Epoch 12/100
4/4 [==============================] - 0s 18ms/step - loss: 0.0224
Epoch 13/100
4/4 [==============================]

In [8]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

plot_data = [
    go.Scatter(
        x=hist['epoch'],
        y=hist['loss'],
        name='loss'
    )
    
]

plot_layout = go.Layout(
        title='Training loss'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [9]:
pred_list = []

batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range(n_input):   
    pred_list.append(model.predict(batch)[0]) 
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

1/1 [==============================] - 0s 18ms/step


In [10]:
add_dates = [df.index[-1] + DateOffset(months=x) for x in range(0,13) ]
future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)

In [11]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=future_dates[-n_input:].index, columns=['Prediction'])

df_proj = pd.concat([df,df_predict], axis=1)

df_proj.tail(12)

,Sales,Prediction
1904-01-01,NaN,635.631394
1904-02-01,NaN,663.875773
1904-03-01,NaN,706.316631
1904-04-01,NaN,745.780924
1904-05-01,NaN,793.688056
1904-06-01,NaN,848.417397
1904-07-01,NaN,906.074270
1904-08-01,NaN,954.205647
1904-09-01,NaN,1036.244822
1904-10-01,NaN,1092.761955


In [12]:
plot_data = [
    go.Scatter(
        x=df_proj.index,
        y=df_proj['Sales'],
        name='actual'
    ),
    go.Scatter(
        x=df_proj.index,
        y=df_proj['Prediction'],
        name='prediction'
    )
]

plot_layout = go.Layout(
        title='Shampoo sales prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)